In [ ]:
import tensorflow as tf 
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Dropout
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from keras.layers.core import Dense
import numpy as np
import matplotlib.pyplot as plt
from physionet_processing import (fetch_h5data, spectrogram, 
                                  special_parameters, transformed_stats)

from physionet_generator import DataGenerator
from scipy import signal

In [ ]:
# X = pickle.load(open("/Users/sanda/Documents/ptb_xl/ptb_xl_v2_pcl/all_X_ptbxl.pcl", "rb"))
# y = pickle.load(open("/Users/sanda/Documents/ptb_xl/ptb_xl_v2_pcl/all_y_ptbxl.pcl", "rb"))

# X = pickle.load(open("/Users/macbookpro/Documents/ptb_xl/scp_filter_v2/all_X_ptbxl.pcl", "rb"))
# y= pickle.load(open("/Users/macbookpro/Documents/ptb_xl/scp_filter_v2/all_y_ptbxl.pcl", "rb"))

X = pickle.load(open("/scratch/thurasx/ptb_xl/scp_filter_v2/all_X_ptbxl.pcl", "rb"))
y= pickle.load(open("/scratch/thurasx/ptb_xl/scp_filter_v2/all_y_ptbxl.pcl", "rb"))


In [ ]:
print(X.shape)

In [ ]:
#comvert X to
# Maximum sequence length
max_length = 1000
# Output dimensions
sequence_length = max_length
spectrogram_nperseg =32 # Spectrogram window
spectrogram_noverlap = 16 # Spectrogram overlap
batch_size = 32
log_spectrogram=True

fs = 300
f, t, Sxx = signal.spectrogram(X[:,:,1], fs=fs, nperseg=spectrogram_nperseg, noverlap=spectrogram_noverlap)
# Sxx = np.transpose(Sxx,[0,2,1])
if log_spectrogram:
    Sxx = abs(Sxx) # Make sure, all values are positive before taking log
    mask = Sxx > 0 # We dont want to take the log of zero
    Sxx[mask] = np.log(Sxx[mask])



In [ ]:
print(Sxx.shape)
plt.imshow(Sxx[0])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Sxx, y, random_state=1)
layer = tf.keras.layers.CategoryEncoding(num_tokens=5, output_mode="one_hot")
y_train = layer(y_train.astype(np.int32))

In [ ]:
#define model
model = Sequential()
model.add(Conv2D(256, (5,5), input_shape=(17,61,1)))
model.add(Activation('relu'))
# model.add(MaxPool2D(pool_size=(5,1)))

model.add(Conv2D(128, (4,4)))
model.add(Activation('relu'))
# model.add(MaxPool2D(pool_size=(4,1)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,1)))

model.add(Flatten())
model.add(Dense(10))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

model.summary()

In [ ]:
epoch = 100
validation_step = 100
batch_size = 100
history = model.fit(X_train, y_train, validation_split=0.2, epochs=epoch, validation_steps=validation_step, verbose=1)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['accuracy'], label = "accuracy")
plt.plot(history.history['val_accuracy'], label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
pred = model.predict(X_test)
pred_nohe = np.array([i.argmax() for i in pred])
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test.astype(np.int32), pred_nohe))
print(f'The accuracy of the model is {accuracy_score(y_test.astype(np.int32), pred_nohe )}')
disp.plot()